# Creating predictions file

In [ ]:
#prediction file from finetuning
import json
with open('Electra_analysis/electra/models/electra_large/results/squad_qa/quac_preds.json', 'r') as JSON:
    #or change the path to where the corresponding files can be found
        json_dict = json.load(JSON)

In [ ]:
import pandas as pd
#turning into json into dataframe
x = []
for d in json_dict.items():
    x.append(d)
df = pd.DataFrame(x)
df.columns = ['id','answer_predicted']
df.to_csv('ans.csv',index=False)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
with open('Electra_analysis/electra/models/electra_large/results/squad_qa/quac_null_odds.json', 'r') as JSON:
        #or change the path to where the corresponding files can be found
        json_dict = json.load(JSON)

In [ ]:
x = []
for d in json_dict.items():
    x.append(d)
null = pd.DataFrame(x)
null.columns = ['id','null_ods']


In [ ]:
null.head()

In [ ]:
no_ans= pd.merge(null, df, on='id')

In [ ]:
for i in range(len(no_ans)):
    if float(no_ans.iloc[i,1])> -1.1201493740081787:  #threshold picked after finetuning that maximuized F1
        no_ans.iloc[i,2]=''

In [ ]:
no_ans.head()

In [ ]:
no_ans.tail()

In [ ]:
js=dict(zip(no_ans.id, no_ans.answer_predicted))

In [ ]:
with open('predictions.json', 'w') as fp:
    json.dump(js, fp)

# Changing format of original datasets 

In [ ]:
#turning original datasets into usable files for analysis
import re
import  string
devset_path = "Electra_analysis/electra/finetuning_data/squad/quac_dev_w.json"
trainset_path = "Electra_analysis/electra/finetuning_data/squad/quac_train_w.json"

pred_path = "predictions.json"

def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
        return re.sub(regex, ' ', text)
    def white_space_fix(text):
        return ' '.join(text.split())
    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)
    def lower(text):
        return text.lower()
    return white_space_fix(remove_articles(remove_punc(lower(s))))

with open(trainset_path) as f:
    train = json.load(f)
    train_data = train['data']

with open(devset_path) as f:
    dev = json.load(f)
    dev_data = dev['data']

with open(pred_path) as f:
    pred = json.load(f)

has_answer_results = []
no_answer_results = []
for data_blob in dev_data:
    paragraphs = data_blob['paragraphs']
    for paragraph in paragraphs:
        for qa in paragraph['qas']:
            pred_answer = normalize_answer(pred[qa['id']])
            correct = False
            if qa['is_impossible']:
                if pred_answer=='':
                    correct = True
                    pred_answer = '-'
                no_answer_results.append('{}\t{}\t{}\t{}\t{}\t{}\t{}\t'.format(qa['id'], correct, pred_answer, qa['question'], paragraph['context'], qa['followup'],qa['yesno']))
            else:
                all_answers = []

                for answer_dict in qa['answers']:
                    answer = normalize_answer(answer_dict['text'])
                    all_answers.append(answer)
                    correct = correct or answer == pred_answer
                if not pred_answer:
                    pred_answer = '-'
                all_answers = '['+'|'.join(all_answers)+']'
                has_answer_results.append('{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t'.format(paragraph['context'], qa['id'], correct, pred_answer, all_answers, qa['question'], qa['followup'],qa['yesno']))

has_answer_results = sorted(has_answer_results, key=lambda x:x.split('\t')[2])
no_answer_results = sorted(no_answer_results, key=lambda x:x.split('\t')[1])
with open("elect-question-type-has-answer-stats.txt", 'w',encoding="utf8") as f:
    f.write('{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t\n'.format('context','id','TF','pred_answer','possible_answers','question','followup','yesno'))
    f.write('\n'.join(has_answer_results))

with open("elect-question-type-no-answer-stats.txt", 'w',encoding="utf8") as f:
    f.write('{}\t{}\t{}\t{}\t{}\t{}\t{}\t\n'.format('id','TF','answer','question','context','followup','yesno'))
    f.write('\n'.join(no_answer_results))

# Creating csv file out of unanswerable questions

In [ ]:
import pandas as pd
df_n = pd.read_fwf('elect-question-type-no-answer-stats.txt', delimiter='\t')


In [ ]:
df_n.head()

In [ ]:
import csv
with open ('elect-question-type-no-answer-stats.txt', 'r',encoding="utf8") as f:
    first_column_n = [column for column in csv.reader(f,delimiter='\t')]
    

In [ ]:
from pandas import DataFrame
df_n = DataFrame (first_column_n,columns=first_column_n[0])


In [ ]:
df_n = df_n.iloc[1:]
df_n.head()

In [ ]:
df_n = df_n[df_n.TF != '']

In [ ]:
df_n.to_csv('no_answer_quac.csv')

# Creating csv file out of answerable questions

In [ ]:
import csv
with open ('elect-question-type-has-answer-stats.txt', 'r', encoding="utf8") as f:
    first_column_y = [column for column in csv.reader(f,delimiter='\t')]
    

In [ ]:
from pandas import DataFrame
df_y = DataFrame (first_column_y,columns=first_column_y[0])


In [ ]:
df_y = df_y.iloc[1:]
df_y.head()

In [ ]:
#df_y = df_y[df_y.TF != ''] just in case where there would be something else in the TF column

In [ ]:
df_y.to_csv('has_answer_quac.csv')